<center>
<h1><b>Word Embedding Based Answer Evaluation System for Online Assessments (WebAES)</b></h1>
<h3>A smart system to automate the process of answer evaluation in online assessments.</h3>
<h5>Doc2Vec Model training</h5>

# Environment setup

For this project the following packages and libraries are required:

1. **string:** To perform string manipulation required for basic text pre-processing.
2. **gensim:** Contains the Doc2Vec model and other founctions for building, training and saving a Doc2Vec model.
3. **scipy:** Contains implmentations of mathematical functions such as cosine of angle between 2 vectors.
4. **time:** To measure the time elapsed for model training.

These packages and libraries are installed and imported in the following code cell.

In [1]:
# Install and import all required packages

# For string manipulation
import string

# To build, train and save a Doc2Vec model
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim.downloader as api
from gensim.test.utils import get_tmpfile

# To determine similarity
import scipy

# To measure time elapsed
import time

# Supress warnings
import warnings
warnings.filterwarnings('ignore')

# Doc2Vec Model

## 1. Training corpus

To build and train a word/document embedding model using Doc2Vec, a training corpus of text documents is required. One such corpus is the **text8 corpus**, which contains the first **10^8 bytes (100MB)** of data from the **English Wikipedia** dump. Therefore, it contains a large number of English documents sourced from Wikipedia, that talk about a number of varied topics. Such a text corpus is suitable to build a Doc2Vec model that can form vector representations of documents and words.

In the following code cell, the text8 corpus, already available in the gensim library, is loaded and a sample document is displayed.

In [2]:
# Load text8 corpus
text8_corpus = api.load('text8')
text8_data = [doc for doc in text8_corpus]

# Display a sample document
print(text8_data[1][0:25])

['reciprocity', 'qualitative', 'impairments', 'in', 'communication', 'as', 'manifested', 'by', 'at', 'least', 'one', 'of', 'the', 'following', 'delay', 'in', 'or', 'total', 'lack', 'of', 'the', 'development', 'of', 'spoken', 'language']


In order to train a Doc2Vec model, a list of Tagged Documents is required. The Doc2Vec model takes this list of Tagged Documents as input and generates vector representations of texts (words and documents) by learning how different words are used along with other words. This makes it possible to capture semantic relationships between words and thereby, capture the meaning of a document (set of words).

In the following code cell, a function is defined to generate Tagged Documents using the *TaggedDocument()* function available in the gensim library. Each document in the text corpus is plit into its individual words (tokens) and tagged with a unique ID. A sample Tagged Document is shown.

In [3]:
# Function to generate tagged documents from text corpus
def tagged_document(corpus_documents):
    # For each document in corpus, yield a TaggedDocument object
    for i, list_of_words in enumerate(corpus_documents):
        yield TaggedDocument(list_of_words, [i])

In [4]:
# Get tagged documents for training data
training_data = list(tagged_document(text8_data))

# Display a sample tagged document
print(training_data[1])

TaggedDocument(['reciprocity', 'qualitative', 'impairments', 'in', 'communication', 'as', 'manifested', 'by', 'at', 'least', 'one', 'of', 'the', 'following', 'delay', 'in', 'or', 'total', 'lack', 'of', 'the', 'development', 'of', 'spoken', 'language', 'not', 'accompanied', 'by', 'an', 'attempt', 'to', 'compensate', 'through', 'alternative', 'modes', 'of', 'communication', 'such', 'as', 'gesture', 'or', 'mime', 'in', 'individuals', 'with', 'adequate', 'speech', 'marked', 'impairment', 'in', 'the', 'ability', 'to', 'initiate', 'or', 'sustain', 'a', 'conversation', 'with', 'others', 'stereotyped', 'and', 'repetitive', 'use', 'of', 'language', 'or', 'idiosyncratic', 'language', 'lack', 'of', 'varied', 'spontaneous', 'make', 'believe', 'play', 'or', 'social', 'imitative', 'play', 'appropriate', 'to', 'developmental', 'level', 'restricted', 'repetitive', 'and', 'stereotyped', 'patterns', 'of', 'behavior', 'interests', 'and', 'activities', 'as', 'manifested', 'by', 'at', 'least', 'one', 'of',

## 2. Initialise and train Doc2Vec model

Having loaded the text8 corpus and generated Tagged Documents for each document, the next step is to initialise and train the Doc2Vec model.

For the purpose of this project, the following parameters are selected for the Doc2Vec model:

1. **vector_size = 50** (the model will generate a 50-dimensional vector for each document)
2. **min_count = 2** (words that occur less than 2 times in the text corpus will be ignored)
3. **epochs = 40** (the model will train iteratively for 40 epochs)

First, the vopcabulary for the model is built based on the text8 corpus. This includes all the words that occur more the twice in the corpus. Next, the model is trained using the training data (corpus). Total time elapsed for this process (in seconds) is shown.

In [5]:
# Get start time
start = time.time()

# Initialise Doc2Vec model
model = Doc2Vec(vector_size=50, min_count=2, epochs=40)

# Build vocabulary from given text corpus
model.build_vocab(training_data)

# Train model
model.train(training_data, total_examples=model.corpus_count, epochs=model.epochs)

# Get end time
end = time.time()

# Display time elapsed
print('Time elapsed: {} seconds'.format(end-start))

Time elapsed: 341.5565505027771 seconds


The Doc2Vec model takes nearly 5 minutes to train.

Once the Doc2Vec model is trained, it can be used to infer/generate the vector representations for any peice of text, and thereby determine the similarity between pairs of texts using cosine similarity measure. Some examples are shown below.

In [6]:
# Sample texts
sample_text1 = 'Nothing is bad'
sample_text2 = 'Everything is good'

# Infer vectors for sample texts using trained Doc2Vec model
sample_text1_vector = model.infer_vector(sample_text1.split())
sample_text2_vector = model.infer_vector(sample_text2.split())

# Determine similarity using cosine similarity
sim = 1 - scipy.spatial.distance.cosine(sample_text1_vector, sample_text2_vector)

# Display similarity score
print(sim)

0.9391225576400757


Using the described method, the similarity between the 2 sample texts is determined to be 0.9391 (~94%).

## 3. Test with sample documents

After training the Doc2Vec model, it can be tested with a pair of sample documents to determine semantic similarity between them.

The function defined below takes a text document as input and tokenizes it (split into individual words) after removing unwanted punctuations. It return a list of tokens for the input document.

In [7]:
# Function to tokenize text documents
def tokenize(document):
    # Remove all punctuation marks
    document = document.translate(str.maketrans('', '', string.punctuation))
    
    # Split document into individual words
    tokens = document.lower().split()
    
    # Return list of tokens
    return tokens

Sample document 1 (source: https://www.datarobot.com/wiki/data-science/)

The vector representation for the sample document is inferred using the trained Doc2Vec model and displayed here.

In [8]:
sample_doc1 = '''Data science is the field of study that combines domain expertise, programming skills, and knowledge of 
mathematics and statistics to extract meaningful insights from data. Data science practitioners apply machine learning 
algorithms to numbers, text, images, video, audio, and more to produce artificial intelligence (AI) systems to perform 
tasks that ordinarily require human intelligence. In turn, these systems generate insights which analysts and business 
users can translate into tangible business value.'''

# Tokenize sample document
sample_doc1_tokens = tokenize(sample_doc1)

# Infer vector using trained model
doc1_vector = model.infer_vector(sample_doc1_tokens)

# Display inferred vector
print(doc1_vector)

[-0.16482636 -0.74337775 -1.5646675   0.8694957   0.09311173  0.17872255
 -0.04526307 -0.7067927  -0.84735596  0.02986916  0.13655894  1.1963602
  0.03421703  0.27110845  0.581937   -2.149735    0.119889   -0.0979569
 -1.6302047  -2.1815634  -1.4354366   0.02248219 -1.4496957  -1.1307305
 -0.9747813  -1.7597828  -0.94408476 -0.43686312 -0.7386884  -0.22812304
  1.5281913   0.35421208 -0.71916246 -0.33932716  0.00802663  0.0655841
  1.2571244   0.02880902  0.02185997  1.3123223   0.8202474  -0.8436277
  0.28407213 -1.4710335   1.1642599  -0.5580641  -0.23785609 -0.8125261
  1.2437353  -0.7836036 ]


Sample document 2 (source: https://en.wikipedia.org/wiki/Data_science)

The vector representation for the sample document is inferred using the trained Doc2Vec model and displayed here.

In [9]:
sample_doc2 = '''Data science is an interdisciplinary field that uses scientific methods, processes, algorithms and 
systems to extract knowledge and insights from noisy, structured and unstructured data, and apply knowledge and 
actionable insights from data across a broad range of application domains. Data science is related to data mining, 
machine learning and big data.'''

# Tokenize sample document
sample_doc2_tokens = tokenize(sample_doc2)

# Infer vector using trained model
doc2_vector = model.infer_vector(sample_doc2_tokens)

# Display inferred vector
print(doc2_vector)

[-0.23594588 -1.1666204  -1.2899795   0.80111146 -0.18388018  0.58754414
 -0.21507066 -0.08047089 -0.13007504  1.233604   -0.40841463  0.81664014
  0.25892878 -0.8679798  -0.05978354 -1.4475968  -0.04929357  0.40120476
 -1.1139549  -1.9979146  -1.4510673   0.90939516 -1.1482767  -0.7303175
 -0.9978392  -1.0596384  -0.19142725 -0.7888661  -0.05551424 -0.26047173
  1.3924643   0.5587637   0.4529767   0.19193275 -0.3496314   0.6942338
  1.2052267  -0.05613485 -0.32192615  0.9985134   0.69988286 -0.72092396
 -0.6347681  -1.5286274   0.6500923  -0.4055486  -0.08273704 -0.6091171
  1.2124068  -0.9821702 ]


Cosine of the angle between these 2 vectors can be used to determine the measure of similarity between the 2 documents.

The cosine of the angle between 2 vectors gives a measure of how close the 2 vectors are to each other in the vector space. The closer 2 vectors are to each other, smaller will be the value of cosine of the angle between them. Subtracting this value from 1 gives a measure of the similarity (closeness) of the pair of vectors.

In [10]:
# Determine similarity score using cosine similarity
sim_score = 1 - scipy.spatial.distance.cosine(doc1_vector, doc2_vector)

# Display score
print(sim_score)

0.8485517501831055


The similarity score for the given sample documents, as calculated using cosine similarity, is 0.8485. It can be said that the 2 documents are ~85% similar.

# Save trained model

This trained Doc2Vec model can be saved for later use.

In [11]:
# Save model to disk
model.save('./WebAES_Doc2Vec_Model.model')